# Periodic problems and plane-wave discretisations

Before digging into big real-world problems, i.e. solving density-functional theory in 3D, we will first take a step back and briefly look at some specifics of periodic problems and plane-wave discretisations.

To make things simple, we first stay in 1D where things are nice and gentle ...

## Lattices

### The Bloch transform

- Consider a 1D periodic **lattice** with lattice constant $a$:
  ```
    |-------|-------|-------|
        a       a       a
  ```
- We take its **unit cell** as the interval $A = [-a/2, a/2)$
- Consider free-electron **Hamiltonian** (== a differential operator)
  $$ H = -\frac12 \Delta $$
  and we are interested in seeking its eigenfunctions $ H \psi_i = \varepsilon_i \psi_i $.
  
  
- The free-electron model is a model where electrons can freely move and there motion
  in only governed by their own kinetic energy.
  The Hamiltonians for solving density-functional theory (our ultimte goal)
  is more involved, but many ideas we will discuss here carry forward.

- Perodicity of the problem implies that the Hamiltonian commutes with lattice translations $T_{n a}$
  $$ T_{n a} H = H T_{n a} \qquad \forall n \in \mathbb{Z}$$
  
  
- In this case **Bloch's theorem** says the solution has the form $ \psi = e^{i k \cdot x} u_k(x) $
  where $u_k(x)$ has the perodicity of the lattice
  $$ u_k(x + na) = u_k(x) \qquad \forall n \in \mathbb{Z}$$
  and is indexed in a real number $k$ (more on this in a sec).
  
  
- Apply $-\Delta = -\frac{d^2}{dx^2}$ to this ansatz:
  $$
    -\Delta \left(e^{i k \cdot x} u_k(x)\right) = (k^2 - 2ik \nabla - \Delta) ) e^{i k \cdot x} u_k(x) = (-i \nabla + k)^2 e^{i k \cdot x} u_k(x)
  $$
  
  
- With this in mind it is natural to define a family
  $$ H_k = \frac12 (-i \nabla + k)^2 $$
  of $k$-specific blocks of $H$.

- In fact one can find a unitary transformation $U$ (the **Bloch transform**) (represented as a familiy of unitary operators $U_k$) such that for any $k$:
$$ (U_k H U_k^{-1}) = H_k. $$


- This is a remarkable result, because it tells us that for seeking all eigenpairs of $H$ we actually only need to seek all eigenpairs of $H_k$ and concatenate them.

### The Brillouin zone

The big mystery in the discussion so far is the parameter $k$ of the operator families.

- In principle $k$ can take *any* real value. It turns out, however, that some of these values will give rise
  to operators related by unitary transformations
- Since such operators have the same eigenspectrum, only one version needs to be considered.
- The smallest subset from which $k$ is chosen is the **Brillouin zone** (BZ).


- The BZ is the unit cell of the **reciprocal lattice**, which may be constructed from the **real-space lattice**
  by a Fourier transform.
- In our simple 1D case the reciprocal lattice is just
  ```
    |--------|--------|--------|
       2π/a     2π/a     2π/a
  ```
  i.e. like the real-space lattice, but just with a different lattice constant $b = 2\pi / a$.
- The BZ in this example is $B = [-π/a, π/a)$

### Discretising the problem

- To solve the problem, we just need to find the eigenspectrum of all $H_k$ with $k \in B$


- **Problem 1:** $B$ is an overcountable set $\Rightarrow$ **$k$-point sampling**: Instead of considering the full Brillouin zone we just consider a regular grid of values for $k$, the so-called **$k$-points**


- **Problem 2:** Each $H_k$ is an operator over an infinite-dimensional domain $\Rightarrow$ Project the problem into a finite basis diagonalise the resulting matrix (Ritz-Galerkin ansatz).

### Plane-wave basis sets

Multiple basis sets are used in practice. One basis set agreeing particularly well with the nature of periodic problems
are plane waves. For our 1D example normalised plane waves are defined as the functions
$$ e_{G}(x) = \frac{e^{i G x}}{\sqrt{a}}  \qquad G \in b\mathbb{Z} $$
and typically one forms basis sets from these by specifying a so-called **kinetic energy cutoff** $E_\text{cut}$:
$$ \left\{ e_{G} \, \big| \, (G + k)^2 \leq 2E_\text{cut} \right\} $$

Notice that (our) $H_k$ is already diagonal with this basis:
\begin{align}
\langle e_{G} | H_k e_{G'} \rangle
&=  \frac12 \left\langle e_{G} \middle| (-i \nabla + k)^2 \, e_{G'} \right\rangle \\
&= \frac12 (G + k)^2 \left\langle e_{G} \middle|e_{G'}\right\rangle \\
&= \frac12 (G + k)^2 \delta_{GG'}
\end{align}

## Let's crunch some numbers!

One typical approach to get physical insight into a Hamiltonian $H$ is to plot a so-called **band structure**, that is the eigenvalues of $H_k$ versus $k$. The rough steps to plot a band structure are:

1. Select a set of $k$-points ($k$-point sampling)
1. Select plane-wave cutoff, thus the basis for discretising $H_k$
1. Build and diagonalise $H_k$ for each $k$.
1. Plot eigenvalues versus $k$

In [ ]:
using LinearAlgebra
using Plots

# Parameters
a = 100     # Lattice constant
Ecut = 300  # in Hartree
kgrid = 50  # Number of points on equally-spaced grid for k

# Derived quantities
b = 2π / a  # Reciprocal lattice

# Step 1: k-Points
kpoints = b * (collect(1:kgrid) .- ceil(Int, kgrid / 2)) ./ kgrid

# Step 2: Basis for H_k
#         Represented as one array of all valid G*b per kpoint
Gmax = ceil(Int, sqrt(2Ecut) + b)  # Rough upper bound for G
Gs = [[Gidx*b for Gidx in -Gmax:Gmax if abs2(Gidx*b + k) ≤ 2Ecut]
      for k in kpoints]

# Step 3: Build the discretised Hk. In this case it is diagonal,
#         i.e. its diagonal values (== eigenvalues) are all we need.
#         We directly determine them and sort them ascendingly
ev_Hk = [sort([abs2(G + k)/2 for G in Gs[ik]])
         for (ik, k) in enumerate(kpoints)]

# Step 4: Plot the bands
n_bands = 6
bands = [[ev_Hk[ik][iband] for ik in 1:length(kpoints)]
         for iband in 1:n_bands]

p = plot()
for iband in 1:n_bands
    plot!(p, kpoints, bands[iband], color=:blue, label="")
end
p

Exactly the same can be achieved using DFTK, albeit in essentially only four lines of code:

In [ ]:
using DFTK
using Unitful
using UnitfulAtomic
using Plots
using LinearAlgebra

# Build the 1D lattice. DFTK is mostly tailored for 3D problems.
# Therefore quantities related to the problem space are have a fixed
# dimension 3. The convention is that for 1D / 2D problems the
# tailling entries are always zero and ignored in the computation.
# For the lattice we therefore construct a 3x3 matrix with only one entry.
lattice = zeros(3, 3)
lattice[1, 1] = 100.

# Select a model. In this case we choose a free-electron model,
# which is the same as saying that there is only a Kinetic term
# (and no potential) in the model. The `n_electrons` is dummy here.
model = Model(lattice; n_electrons=0, terms=[Kinetic()])
@show model.n_dim

# Step 2: Define a plane-wave basis using this model and the same cutoff
# as before. The kpoint grid is given as a regular grid in the BZ.
# Here we select only one k-Point (1x1x1). The kpoints for calculating and plotting
# the bands are selected below. (This distinction between k-Points for the basis
# and the plotting is needed for more complicated models like DFT)
basis = PlaneWaveBasis(model; Ecut=300, kgrid=(1, 1, 1));

# Steps 1, 3 & 4: K-Point selection, discretisation, diagonalisation
n_bands = 6
ρ0 = guess_density(basis)  # Just dummy, has no meaning in this model
p  = plot_bandstructure(basis, ρ0, n_bands, kline_density=15, unit=u"hartree")

## Adding potentials

So far so good. But free electrons are actually a little boring,
so let's add a potential interacting with the electrons.

- The modified problem we will look at consists of diagonalising

$$ H_k = \frac12 (-i \nabla + k)^2 + V $$

for all $k \in \text{BZ}$ with a periodic potential $V$ interacting with the electrons.

- A number of "standard" potentials are readily implemented in DFTK and
  can be assembled using the `terms` kwarg of the model.
  This allows to seamlessly construct
  
    * density-functial theory models for treating electronic structures
      (see the next notebook)
    * Gross-Pitaevskii models for bosonic systems
    * even some more unusual cases like anyonic models.  
  
  
In this workbook we will go a little more low-level and directly provide
an analytic potential describing the interaction with the electrons to DFTK.

First we define a custom Gaussian potential as a new "element" inside DFTK:

In [ ]:
using DFTK

struct ElementGaussian <: DFTK.Element
    α  # Prefactor
    L  # Extend
end
# Dummy (needed for cases modelling actual atoms)
DFTK.charge_ionic(el::ElementGaussian) = 0
DFTK.charge_nuclear(el::ElementGaussian) = 0

# Some default values
ElementGaussian() = ElementGaussian(0.3, 10.0)

# Real-space representation of a Gaussian
function DFTK.local_potential_real(el::ElementGaussian, r::Real)
    -el.α / (√(2π) * el.L) * exp(- (r / el.L)^2 / 2)
end

# Fourier-space representation of the Gaussian
function DFTK.local_potential_fourier(el::ElementGaussian, q::Real)
    # = ∫ -α exp(-(r/L)^2 exp(-ir⋅q) dr
    -el.α * exp(- (q * el.L)^2 / 2)
end

A single potential looks like this:

In [ ]:
using Plots
nucleus = ElementGaussian()
plot(r -> DFTK.local_potential_real(nucleus, norm(r)), xlims=(-50, 50))

With this element at hand we can easily construct a setting
where two potentials of this form are located at positions
$20$ and $80$ inside the lattice $[0, 100]$:

In [ ]:
using LinearAlgebra

# Define the 1D lattice [0, 100]
lattice = diagm([100., 0, 0])

# Place them at 20 and 80 in *fractional coordinates*,
# that is 0.2 and 0.8, since the lattice is 100 wide.
nucleus = ElementGaussian()
atoms = [nucleus => [[0.2, 0, 0], [0.8, 0, 0]]]

# Assemble the model, discretise and build the Hamiltonian
model = Model(lattice; atoms=atoms, terms=[Kinetic(), AtomicLocal()])
basis = PlaneWaveBasis(model; Ecut=300, kgrid=(1, 1, 1));
ham   = Hamiltonian(basis)

# Extract the total potential term of the Hamiltonian and plot it
potential = DFTK.total_local_potential(ham)[:, 1, 1]
rvecs = collect(r_vectors_cart(basis))[:, 1, 1]  # slice along the x axis
x = [r[1] for r in rvecs]                   # only keep the x coordinate
plot(x, potential, label="", xlabel="x", ylabel="V(x)")

Notice how DFTK took care of the periodic wrapping of the potential values going from $0$ and $100$.

With this setup, let's look at the bands:

In [ ]:
using Unitful
using UnitfulAtomic

n_bands = 6
ρ0 = zeros(eltype(basis), basis.fft_size..., 1)  # Just dummy, has no meaning in this model
p = plot_bandstructure(basis, ρ0, n_bands, kline_density=15, unit=u"hartree")

The bands are noticably different.
- The bands no longer overlap, meaning that the spectrum of $H$ is no longer continous but has gaps.

- The two lowest bands are almost flat, which means that they represent
  two tightly bound and localised electrons inside the two Gaussians. 
  
- The higher the bands, the more curved they become. In other words the higher the kinetic energy of the electrons the more delocalised they become and the less they feel the effect of the two Gaussian potentials.

**Exercise:** Try playing with the parameters of the Gaussian potentials by setting
```julia
nucleus = ElementGaussian(α, L)
```
with different $\alpha$ and $L$ in the above procedure.

## Going to 3D

- Our example: Silicon

| <img src="img/Silicon_crystal.jpg" width=180 height=180 /> |
| --------------- |
| silicon crystal |

- $A$ becomes a matrix, e.g. for silicon (diamond structure)
  $$ A = \frac{a}{2} \left( \begin{array}{ccc} 0&1&1\\1&0&1\\1&1&0 \end{array} \right) \qquad a = 10.26\,\text{bohr}. $$
- Brillouin zones become more involved:

| <img src="img/Brillouin_Zone.svg" width=250 height=250 /> |
| --------------------------------------------------------- |
|   face-centred cubic BZ (applies to silicon)              |
  
- $k$-points are vectors taken from a rectangular 3D grid inside the BZ
- Plane-wave wave vectors $G$ are vectorial:
  $$ e_G = \frac{1}{\sqrt{\text{det}(A)}} e^{i G\cdot r} $$
- Cutoff taken over norm:
  $$ \{ e_G | \|G + k\|^2 \leq 2E_\text{cut} \} $$

- The bandstructure of silicon:
<img src="img/silicon_bands.png" width=500 height=500 />

#### Takeaway

- For periodic problems the Bloch transform allows to find the eigenpairs of an operator $k$-Point by $k$-Point
- The $k$ points are taken in a discrete mesh from the Brillouin zone.
- A common way to visualise the eigenvalues is as a plot versus $k$, the band structure.